In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision 
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid

In [ ]:
tst=ImageFolder("/kaggle/input/dogs-cats-images/dog vs cat/dataset/training_set/",transform=transforms.ToTensor())

In [ ]:
mean=[]
std=[]
i,j=tst[0]
for x in range(3):
    mean.append(i[x].mean())
    std.append(i[x].std())
    

In [ ]:
print(mean)
print(std)

In [ ]:
trans=transforms.Compose(
    [
        transforms.Resize([32,32]),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ]
)

In [ ]:
dta=ImageFolder("/kaggle/input/dogs-cats-images/dog vs cat/dataset/training_set/",transform=trans)

In [ ]:
train,vali=torch.utils.data.random_split(dta,[7000,1000])

In [ ]:
dl=DataLoader(train,shuffle=True,batch_size=128,pin_memory=True,num_workers=2)
vl=DataLoader(vali,shuffle=True,batch_size=128,pin_memory=True,num_workers=2)

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.seq=nn.Sequential(
            nn.Conv2d(3,8,kernel_size=3,padding=1,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(8),
            nn.Conv2d(8,16,kernel_size=3,padding=1,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(16),
            nn.Flatten(),
            nn.Linear(1024,64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64,16),
            nn.ReLU(),
            nn.BatchNorm1d(16),
            nn.Linear(16,1),
            nn.Sigmoid()
        )
    def forward(self,x):
        x=self.seq(x)
        return x

In [ ]:
device="cuda" if torch.cuda.is_available() else "cpu"
cnn=Model()
cnn.to(device)
for i,j in dl:
    i=i.to(device)
    j=j.to(device)
    pred=cnn(i)
    break

In [ ]:
def acc(pred,label):
    pred=pred.round()
    return torch.tensor(torch.sum(pred==label)/len(label))

history=[]

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


def fit(epochs,lr,maxlr,model,loss,dl,vl):
    optim=torch.optim.SGD(model.parameters(),lr=lr,weight_decay=0.01)
    sch=torch.optim.lr_scheduler.OneCycleLR(max_lr=maxlr,epochs=epochs,steps_per_epoch=len(dl),optimizer=optim)
    for epoch in range(epochs):
        for i,j in dl:
            j=j.float()
            i=i.to(device)
            j=j.to(device)
            pred=model(i)
            ls=loss(pred,j)
            ls.backward()
            optim.step()
            sch.step()
            optim.zero_grad()
    
        for i,j in vl:
            j=j.float()
            i=i.to(device)
            j=j.to(device)
            with torch.no_grad():
                pred=model(i)
                ls=loss(pred,j)
                ac=acc(pred,j)
                history.append({"loss":ls,"acc":ac})
                print(f"epoch.no={epoch+1} loss={ls} acc={ac}")
            
            

In [ ]:
lr=1e-7
maxlr=0.01
epochs=100
loss=nn.BCELoss()


In [ ]:
# fit(epochs,lr,maxlr,cnn,loss,dl,vl)

In [ ]:
class Model2(nn.Module):
    def __init__(self):
        super().__init__()
        self.seq=nn.Sequential(
            nn.Conv2d(3,8,kernel_size=3,padding=1,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(8),
            nn.Conv2d(8,16,kernel_size=3,padding=1,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(16),
            nn.Flatten(),
            nn.Linear(1024,64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64,16),
            nn.ReLU(),
            nn.BatchNorm1d(16),
            nn.Linear(16,2)
            
        )
    def forward(self,x):
        x=self.seq(x)
        return x

In [ ]:
model=Model2()
loss=nn.CrossEntropyLoss()

In [ ]:
for i,j in dl:
    pred=model(i)
    ls=loss(pred,j)
    break

In [ ]:
def fit2(epochs,lr,maxlr,model,loss,dl,vl):
    optim=torch.optim.SGD(model.parameters(),lr=lr,weight_decay=0.01)
    sch=torch.optim.lr_scheduler.OneCycleLR(max_lr=maxlr,epochs=epochs,steps_per_epoch=len(dl),optimizer=optim)
    for epoch in range(epochs):
        for i,j in dl:
            j=j.long()
            i=i.to(device)
            j=j.to(device)
            pred=model(i)
            ls=loss(pred,j)
            ls.backward()
            optim.step()
            sch.step()
            optim.zero_grad()
    
        for i,j in vl:
            j=j.long()
            i=i.to(device)
            j=j.to(device)
            with torch.no_grad():
                pred=model(i)
                ls=loss(pred,j)
                ac=accuracy(pred,j)
                history.append({"loss":ls,"acc":ac})
                print(f"epoch.no={epoch+1} loss={ls} acc={ac}")
            

In [ ]:
model=Model2()
loss=nn.CrossEntropyLoss()
lr=1e-7
maxlr=0.01
epochs=100

In [ ]:
fit2(epochs,lr,maxlr,model,loss,dl,vl)

In [ ]:
ls=[]
acc=[]
for lst in range(len(history)):
    ls.append(history[lst]["loss"])
    acc.append(history[lst]["acc"])

In [ ]:
plt.plot(ls)